In [ ]:
#names = ['Isabela', 'Juliano', 'Alberto ', 'Gabriela', 'Mi']
#long_names =  [name for name in names if len(name)>4]
#short_names = [x for x in names if len(x)<4]
#contacts = {'Isabela':'0', 'Juliano':'1', 'Alberto ':'4', 'Gabriela':'45', 'Mi':'87'}
#for name, index in contacts.items():
#  print(name,index)

#import os
#home_items = os.listdir('/Users/alfredo')
#home_content = {"files": [], "directories": []}
#home_paths = [os.path.join('/Users/alfredo',item) for item in home_items]
#for path in home_paths:
#  if os.path.isdir(path):
#    home_content["directories"].append(path)
#  else:
#    home_content["directories"].append(path)

#items = ['file', 1,2,3,3,5]
#items.index("file")
#with open("populate.sql") as sql_file:
#  sql_contents = sql_file.readlines()

#import json
#data = {'valid': True}
#with open("simple_data.json", "w") as json_file:
#  json.dump(data,json_file)

In [ ]:
#!pip install --upgrade systemml
!pip install --upgrade https://github.com/niketanpansare/future_of_data/raw/master/systemml-1.1.0-SNAPSHOT-python.tar.gz
!ln -s -f ~/.local/lib/python2.7/site-packages/systemml/systemml-java/*.jar ~/data/libs/

In [ ]:
!pip install findspark
!pip install pyspark
!pip install systemml

In [3]:
%matplotlib inline


Load diabetes dataset from scikit-learn

In [4]:
from sklearn import datasets
import numpy as np


# Load the diabetes dataset
diabetes = datasets.load_diabetes()
diabetes = datasets.load_diabetes()
diabetes_X = diabetes.data[:,np.newaxis,2]
diabetes_X_train =diabetes_X[:-20]
diabetes_X_test = diabetes_X[-20:]
diabetes_y_train = np.matrix(diabetes.target[:-20]).T
diabetes_y_test = np.matrix(diabetes.target[-20:]).T

In [11]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()


In [12]:
from systemml import MLContext, dml
ml = MLContext(spark)
print(ml.info())


Archiver-Version: Plexus Archiver
Artifact-Id: systemml
Build-Jdk: 1.8.0_111
Build-Time: 2018-01-11 16:39:06 CST
Built-By: biuser
Created-By: Apache Maven 3.0.5
Group-Id: org.apache.systemml
Main-Class: org.apache.sysml.api.DMLScript
Manifest-Version: 1.0
Minimum-Recommended-Spark-Version: 2.1.0
Version: 1.1.0-SNAPSHOT



In [21]:
script = """
ones = matrix(1, rows = nrow(X), cols=1)
X = cbind(X, ones)
A = t(X) %*% X
b = t(X) %*% y
w = solve(A, b)
bias = as.scalar(w[nrow(w), 1])
w = w[1:nrow(w)-1, ]
"""
prog = dml(script).input(X=diabetes_X_train, y=diabetes_y_train).output('w', 'bias')
try:
    #w, bias = ml.execute(prog).get('w','bias')
    #w = result.get('w').toNumPy()
    result = ml.execute(prog)
    w = result.get('w').toNumPy()
    bias = result.get('bias')
except Exception as e:
    print("Error during script execution:", e)


SystemML Statistics:
Total execution time:		0.002 sec.
Number of executed Spark inst:	2.


